# Model 9: Learning Rate Sweep

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 8
n_classes = 20
n_epochs = 30
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1

In [6]:
# defining the model architecture
m9_t1 = models.Sequential()

# convolution/max pool stacks
m9_t1.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m9_t1.add(layers.LeakyReLU(alpha=0.1))
m9_t1.add(layers.MaxPooling2D((2,2)))

m9_t1.add(layers.Conv2D(64,(3,3), padding='same'))
m9_t1.add(layers.LeakyReLU(alpha=0.1))
m9_t1.add(layers.MaxPooling2D((2,2)))

m9_t1.add(layers.Conv2D(128,(3,3), padding='same'))
m9_t1.add(layers.LeakyReLU(alpha=0.1))
m9_t1.add(layers.MaxPooling2D((2,2)))

m9_t1.add(layers.Conv2D(256,(3,3), padding='same'))
m9_t1.add(layers.LeakyReLU(alpha=0.1))
m9_t1.add(layers.MaxPooling2D((2,2)))

m9_t1.add(layers.Conv2D(512,(3,3), padding='same'))
m9_t1.add(layers.LeakyReLU(alpha=0.1))
m9_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m9_t1.add(layers.Flatten())
m9_t1.add(layers.Dense(1024))
m9_t1.add(layers.LeakyReLU(alpha=0.1))
m9_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m9_t1.summary()
model_names.append('m9_t1')
model_list.append(m9_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)      

### Trial 2

In [7]:
# defining the model architecture
m9_t2 = models.Sequential()

# convolution/max pool stacks
m9_t2.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m9_t2.add(layers.LeakyReLU(alpha=0.1))
m9_t2.add(layers.MaxPooling2D((2,2)))

m9_t2.add(layers.Conv2D(64,(3,3), padding='same'))
m9_t2.add(layers.LeakyReLU(alpha=0.1))
m9_t2.add(layers.MaxPooling2D((2,2)))

m9_t2.add(layers.Conv2D(128,(3,3), padding='same'))
m9_t2.add(layers.LeakyReLU(alpha=0.1))
m9_t2.add(layers.MaxPooling2D((2,2)))

m9_t2.add(layers.Conv2D(256,(3,3), padding='same'))
m9_t2.add(layers.LeakyReLU(alpha=0.1))
m9_t2.add(layers.MaxPooling2D((2,2)))

m9_t2.add(layers.Conv2D(512,(3,3), padding='same'))
m9_t2.add(layers.LeakyReLU(alpha=0.1))
m9_t2.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m9_t2.add(layers.Flatten())
m9_t2.add(layers.Dense(1024))
m9_t2.add(layers.LeakyReLU(alpha=0.1))
m9_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m9_t2.summary()
model_names.append('m9_t2')
model_list.append(m9_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 50, 50, 128)      

### Trial 3

In [8]:
# defining the model architecture
m9_t3 = models.Sequential()

# convolution/max pool stacks
m9_t3.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m9_t3.add(layers.LeakyReLU(alpha=0.1))
m9_t3.add(layers.MaxPooling2D((2,2)))

m9_t3.add(layers.Conv2D(64,(3,3), padding='same'))
m9_t3.add(layers.LeakyReLU(alpha=0.1))
m9_t3.add(layers.MaxPooling2D((2,2)))

m9_t3.add(layers.Conv2D(128,(3,3), padding='same'))
m9_t3.add(layers.LeakyReLU(alpha=0.1))
m9_t3.add(layers.MaxPooling2D((2,2)))

m9_t3.add(layers.Conv2D(256,(3,3), padding='same'))
m9_t3.add(layers.LeakyReLU(alpha=0.1))
m9_t3.add(layers.MaxPooling2D((2,2)))

m9_t3.add(layers.Conv2D(512,(3,3), padding='same'))
m9_t3.add(layers.LeakyReLU(alpha=0.1))
m9_t3.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m9_t3.add(layers.Flatten())
m9_t3.add(layers.Dense(1024))
m9_t3.add(layers.LeakyReLU(alpha=0.1))
m9_t3.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m9_t3.summary()
model_names.append('m9_t3')
model_list.append(m9_t3)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 128)      

### Trial 4

In [9]:
# defining the model architecture
m9_t4 = models.Sequential()

# convolution/max pool stacks
m9_t4.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m9_t4.add(layers.LeakyReLU(alpha=0.1))
m9_t4.add(layers.MaxPooling2D((2,2)))

m9_t4.add(layers.Conv2D(64,(3,3), padding='same'))
m9_t4.add(layers.LeakyReLU(alpha=0.1))
m9_t4.add(layers.MaxPooling2D((2,2)))

m9_t4.add(layers.Conv2D(128,(3,3), padding='same'))
m9_t4.add(layers.LeakyReLU(alpha=0.1))
m9_t4.add(layers.MaxPooling2D((2,2)))

m9_t4.add(layers.Conv2D(256,(3,3), padding='same'))
m9_t4.add(layers.LeakyReLU(alpha=0.1))
m9_t4.add(layers.MaxPooling2D((2,2)))

m9_t4.add(layers.Conv2D(512,(3,3), padding='same'))
m9_t4.add(layers.LeakyReLU(alpha=0.1))
m9_t4.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m9_t4.add(layers.Flatten())
m9_t4.add(layers.Dense(1024))
m9_t4.add(layers.LeakyReLU(alpha=0.1))
m9_t4.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m9_t4.summary()
model_names.append('m9_t4')
model_list.append(m9_t4)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 50, 50, 128)      

### Trial 5

In [10]:
# defining the model architecture
m9_t5 = models.Sequential()

# convolution/max pool stacks
m9_t5.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m9_t5.add(layers.LeakyReLU(alpha=0.1))
m9_t5.add(layers.MaxPooling2D((2,2)))

m9_t5.add(layers.Conv2D(64,(3,3), padding='same'))
m9_t5.add(layers.LeakyReLU(alpha=0.1))
m9_t5.add(layers.MaxPooling2D((2,2)))

m9_t5.add(layers.Conv2D(128,(3,3), padding='same'))
m9_t5.add(layers.LeakyReLU(alpha=0.1))
m9_t5.add(layers.MaxPooling2D((2,2)))

m9_t5.add(layers.Conv2D(256,(3,3), padding='same'))
m9_t5.add(layers.LeakyReLU(alpha=0.1))
m9_t5.add(layers.MaxPooling2D((2,2)))

m9_t5.add(layers.Conv2D(512,(3,3), padding='same'))
m9_t5.add(layers.LeakyReLU(alpha=0.1))
m9_t5.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m9_t5.add(layers.Flatten())
m9_t5.add(layers.Dense(1024))
m9_t5.add(layers.LeakyReLU(alpha=0.1))
m9_t5.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m9_t5.summary()
model_names.append('m9_t5')
model_list.append(m9_t5)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 50, 50, 128)      

## Step 4: Training

In [11]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 326 ms, sys: 1.4 s, total: 1.72 s
Wall time: 1.72 s


In [13]:
# setting up optimizer
opt_1 = optimizers.Adamax(learning_rate=9e-4)
opt_2 = optimizers.Adamax(learning_rate=7e-4)
opt_3 = optimizers.Adamax(learning_rate=5e-4)
opt_4 = optimizers.Adamax(learning_rate=3e-4)
opt_5 = optimizers.Adamax(learning_rate=1e-4)

# compiling loss functions
m9_t1.compile(loss='categorical_crossentropy', optimizer=opt_1, metrics=['acc'])
m9_t2.compile(loss='categorical_crossentropy', optimizer=opt_2, metrics=['acc'])
m9_t3.compile(loss='categorical_crossentropy', optimizer=opt_3, metrics=['acc'])
m9_t4.compile(loss='categorical_crossentropy', optimizer=opt_4, metrics=['acc'])
m9_t5.compile(loss='categorical_crossentropy', optimizer=opt_5, metrics=['acc'])

In [14]:
%%time
# setting up model saving checkpoints
m9_t1_cp = ModelCheckpoint(filepath=f'{model_path}m9_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m9_t1_history = m9_t1.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m9_t1_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m9_t1_history)

Epoch 1/30
1013/1013 [==============================] - 155s 153ms/step - loss: 2.6631 - acc: 0.1504 - val_loss: 1.7678 - val_acc: 0.2475

Epoch 00001: val_acc improved from -inf to 0.24752, saving model to /floyd/home/m9_t1.h5
Epoch 2/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.2922 - acc: 0.2596 - val_loss: 1.6604 - val_acc: 0.3603

Epoch 00002: val_acc improved from 0.24752 to 0.36035, saving model to /floyd/home/m9_t1.h5
Epoch 3/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.1510 - acc: 0.3065 - val_loss: 2.4284 - val_acc: 0.4127

Epoch 00003: val_acc improved from 0.36035 to 0.41272, saving model to /floyd/home/m9_t1.h5
Epoch 4/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.0256 - acc: 0.3479 - val_loss: 1.9253 - val_acc: 0.4227

Epoch 00004: val_acc improved from 0.41272 to 0.42269, saving model to /floyd/home/m9_t1.h5
Epoch 5/30
1013/1013 [==============================] - 79s 78ms/step - loss: 1.9194 

In [15]:
%%time
# setting up model saving checkpoints
m9_t2_cp = ModelCheckpoint(filepath=f'{model_path}m9_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m9_t2_history = m9_t2.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m9_t2_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m9_t2_history)

Epoch 1/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.6328 - acc: 0.1648 - val_loss: 2.5249 - val_acc: 0.2636

Epoch 00001: val_acc improved from -inf to 0.26361, saving model to /floyd/home/m9_t2.h5
Epoch 2/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.2642 - acc: 0.2662 - val_loss: 1.8648 - val_acc: 0.3653

Epoch 00002: val_acc improved from 0.26361 to 0.36534, saving model to /floyd/home/m9_t2.h5
Epoch 3/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.1072 - acc: 0.3194 - val_loss: 1.7667 - val_acc: 0.4015

Epoch 00003: val_acc improved from 0.36534 to 0.40150, saving model to /floyd/home/m9_t2.h5
Epoch 4/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.0044 - acc: 0.3574 - val_loss: 0.8099 - val_acc: 0.4701

Epoch 00004: val_acc improved from 0.40150 to 0.47007, saving model to /floyd/home/m9_t2.h5
Epoch 5/30
1013/1013 [==============================] - 78s 77ms/step - loss: 1.8959 - 

In [16]:
%%time
# setting up model saving checkpoints
m9_t3_cp = ModelCheckpoint(filepath=f'{model_path}m9_t3.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m9_t3_history = m9_t3.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m9_t3_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m9_t3_history)

Epoch 1/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.5884 - acc: 0.1756 - val_loss: 2.8142 - val_acc: 0.2574

Epoch 00001: val_acc improved from -inf to 0.25743, saving model to /floyd/home/m9_t3.h5
Epoch 2/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.2643 - acc: 0.2670 - val_loss: 2.5106 - val_acc: 0.3753

Epoch 00002: val_acc improved from 0.25743 to 0.37531, saving model to /floyd/home/m9_t3.h5
Epoch 3/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.1260 - acc: 0.3073 - val_loss: 2.1820 - val_acc: 0.3304

Epoch 00003: val_acc did not improve from 0.37531
Epoch 4/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.0062 - acc: 0.3448 - val_loss: 2.5495 - val_acc: 0.4539

Epoch 00004: val_acc improved from 0.37531 to 0.45387, saving model to /floyd/home/m9_t3.h5
Epoch 5/30
1013/1013 [==============================] - 78s 77ms/step - loss: 1.9028 - acc: 0.3896 - val_loss: 1.9529 - val_acc: 

In [17]:
%%time
# setting up model saving checkpoints
m9_t4_cp = ModelCheckpoint(filepath=f'{model_path}m9_t4.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m9_t4_history = m9_t4.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m9_t4_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m9_t4_history)

Epoch 1/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.6278 - acc: 0.1610 - val_loss: 2.4638 - val_acc: 0.2958

Epoch 00001: val_acc improved from -inf to 0.29579, saving model to /floyd/home/m9_t4.h5
Epoch 2/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.2495 - acc: 0.2780 - val_loss: 2.2817 - val_acc: 0.3741

Epoch 00002: val_acc improved from 0.29579 to 0.37406, saving model to /floyd/home/m9_t4.h5
Epoch 3/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.1286 - acc: 0.3198 - val_loss: 1.9328 - val_acc: 0.3753

Epoch 00003: val_acc improved from 0.37406 to 0.37531, saving model to /floyd/home/m9_t4.h5
Epoch 4/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.0438 - acc: 0.3400 - val_loss: 1.7933 - val_acc: 0.4165

Epoch 00004: val_acc improved from 0.37531 to 0.41646, saving model to /floyd/home/m9_t4.h5
Epoch 5/30
1013/1013 [==============================] - 79s 78ms/step - loss: 1.9573 - 

In [18]:
%%time
# setting up model saving checkpoints
m9_t5_cp = ModelCheckpoint(filepath=f'{model_path}m9_t5.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m9_t5_history = m9_t5.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m9_t5_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m9_t5_history)

Epoch 1/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.7282 - acc: 0.1422 - val_loss: 2.3298 - val_acc: 0.2475

Epoch 00001: val_acc improved from -inf to 0.24752, saving model to /floyd/home/m9_t5.h5
Epoch 2/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.4116 - acc: 0.2315 - val_loss: 2.0152 - val_acc: 0.3117

Epoch 00002: val_acc improved from 0.24752 to 0.31172, saving model to /floyd/home/m9_t5.h5
Epoch 3/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.2545 - acc: 0.2767 - val_loss: 1.7269 - val_acc: 0.3616

Epoch 00003: val_acc improved from 0.31172 to 0.36160, saving model to /floyd/home/m9_t5.h5
Epoch 4/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.1805 - acc: 0.2964 - val_loss: 1.6684 - val_acc: 0.3616

Epoch 00004: val_acc did not improve from 0.36160
Epoch 5/30
1013/1013 [==============================] - 78s 77ms/step - loss: 2.1026 - acc: 0.3233 - val_loss: 1.7958 - val_acc: 

In [19]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [20]:
# evaluating models on validation set
for key, value in models_dict.items():
    model = models.load_model(f'{model_path}{key}.h5')
    (val_loss, val_accuracy) = model.evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

102/102 [==============================] - 2s 20ms/step
m9_t1 Val Accuracy: 71.36%
m9_t1 Val Loss: 0.5447
---
102/102 [==============================] - 2s 19ms/step
m9_t2 Val Accuracy: 68.52%
m9_t2 Val Loss: 0.1631
---
102/102 [==============================] - 2s 20ms/step
m9_t3 Val Accuracy: 67.53%
m9_t3 Val Loss: 0.7804
---
102/102 [==============================] - 2s 19ms/step
m9_t4 Val Accuracy: 63.09%
m9_t4 Val Loss: 1.3448
---
102/102 [==============================] - 2s 20ms/step
m9_t5 Val Accuracy: 56.54%
m9_t5 Val Loss: 0.4567
---


In [21]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m9_t1_history saved in /floyd/home/
m9_t2_history saved in /floyd/home/
m9_t3_history saved in /floyd/home/
m9_t4_history saved in /floyd/home/
m9_t5_history saved in /floyd/home/
